In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
# !git clone https://github.com/acse-jy220/SFC-CAE-Ready-to-use

In [ ]:
%cd gdrive/MyDrive/DISSERTATION/SFC-CAE-AdaptiveFinal/

In [ ]:
# !git pull

### Install dependencies

In [ ]:
modulelist = !pip list                         #This is just to check whether the requirements are already installed
if str(modulelist).find("vtk") == -1:          #If vtk is already installed don't start the installation process because it will take a couple minutes to verify that everything is there.
  !pip install -e .

### Import useful functions inside this repo, very important, don't forget!!!

In [ ]:
from sfc_cae import *

## (Example) Get the Flow Past Cylinder data `CG_Mesh_new` (2000 snapshots) 
Try to specify you own `data_dir` !!!!

In [ ]:
# !mkdir FPC_Re3900_CG_new
# !wget https://www.dropbox.com/sh/aid0yv2685nln51/AADmtVChECW_B85M8O2cDmR0a
# !unzip AADmtVChECW_B85M8O2cDmR0a -d './FPC_Re3900_CG_new/'
# !rm -rf AADmtVChECW_B85M8O2cDmR0a

# Specify the folder of the data and load the full_tensors from `vtu` files, or `txt` files, like following, check the folder you specified only contains `vtu` and `txt` files!!!

### specify your own {DATA_DIR} of vtu files here.

In [ ]:
# data_path = 'FPC_Re3900_CG_new/'
# vtu_fields = ['Velocity']

### Specify the `vtu_fields` (a list) of an unadapted mesh to read in, returns:
**full_tensor**: The tensor of all `vtu_fields` you specified for all snapshots in the {data_path}, 3-dimension, the first dimension is number of snapshots in that {data_path}, the second is the number of Nodes in each snapshots, the last is number of components in the `vtu_fields`, for example, in this FPC test case,
two components for `Velocity`, one for `Pressure`. <br> <br>
**coords**: The coordinates of the unadapted mesh Nodes from the `vtu` file, 3-dimension.

In [ ]:
# full_tensor, coords, cells = read_in_files(data_path, vtu_fields = vtu_fields)

For reading tensor from `txt` files, check the `read_in_files` function from `util.py` at https://github.com/acse-jy220/SFC-CAE-Ready-to-use

### Save the full tensor in your `google_drive` for further use.

In [ ]:
# torch.save(full_tensor, '/content/gdrive/MyDrive/FPC_CG_new_velocity.pt')
# torch.save(coords, '/content/gdrive/MyDrive/FPC_CG_new_mesh.pt')

Or download a premade one from the link in the readme

### Load tensor from google drive

In [ ]:
full_tensor = torch.load('./savedtensors/FPC_new_velocity.pt')[1:]
coords = torch.load('./savedtensors/FPC_new_mesh.pt')
cells = torch.load('./savedtensors/FPC_new_mesh_cells.pt')

### Generate space-filling-curves from coords (unadapted) of the `vtu` files

In [ ]:
# ncurves = 20 # specify space-filling curve numbers
# # for DG mesh
# # space_filling_orderings, invert_space_filling_orderings = get_sfc_curves_from_coords(coords, ncurves)

# # for CG mesh, according to Claire's code, we need to input an additional template vtu file to it.
# template_vtu = vtktools.vtu(glob.glob(data_path + '*')[0])
# space_filling_orderings, invert_space_filling_orderings = get_sfc_curves_from_coords_CG(coords, ncurves, template_vtu)

In [ ]:
# #You should probably clean them of duplicates:

# nonduplicates = []


# for i in range(len(space_filling_orderings)):

#   duplicate = False

#   for j in range(len(nonduplicates)):
#     if np.diff(space_filling_orderings[i],nonduplicates[j]) == 0:
#       duplicate = True
  
#   if not duplicate:
#     nonduplicates.append(space_filling_orderings[i])

# space_filling_orderings = nonduplicates

In [ ]:
# #And regenerate the inverted space filling curves:
# nonduplicatesinverse = [np.argsort(i) for i in nonduplicates]

In [ ]:
# # see this in get_sfc_curves_from_coords_CG() func in utils.py, find sparse connectivity for CG meshes.
# ncolm=0
# colm=[]
# findm=[0]
# for nod in range(coords.shape[0]):
#     nodes = template_vtu.GetPointPoints(nod)
#     nodes2 = np.sort(nodes) #sort_assed(nodes) 
#     colm.extend(nodes2[:]) 
#     nlength = nodes2.shape[0]
#     ncolm=ncolm+nlength
#     findm.append(ncolm)

# colm = np.array(colm)
# colm = colm + 1
# findm = np.array(findm)
# findm = findm + 1

In [ ]:
# edge_list = csr_to_edges(findm, colm)

In [ ]:
#Or load previously compiled ones!

import pickle

infile = open("./savedcurves/956differentfpccurves.pickle",'rb')
space_filling_orderings = pickle.load(infile)
infile.close()

infile = open("./savedcurves/956differentfpcicurves.pickle",'rb')
invert_space_filling_orderings = pickle.load(infile)
infile.close()

### Print out the filled connectivity by the space-filling curves.

In [ ]:
# filled_edges_for_sfcs(edge_list, space_filling_orderings)

In [ ]:
#visualize the space-filling curve, only works for the FPC example
# plot_trace_vtu_2D(coords[space_filling_orderings[0]], 25)
countour_plot_vtu_2D(coords[space_filling_orderings[0]], 25, cmap = cmocean.cm.dense_r)

In [ ]:
countour_plot_vtu_2D(coords[space_filling_orderings[930]], 25, cmap = cmocean.cm.dense_r)

### Random split the tensor into training_set: valid_set: test_set = 0.8:0.1:0.1 = 8:1:1

In [ ]:
train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 0.1 
train_index, valid_index, test_index = index_split(train_ratio, valid_ratio, test_ratio, total_num = full_tensor.shape[0])

In [ ]:
full_set, k, b = standardlize_tensor(full_tensor, lower = -1, upper = 1)

In [ ]:
k.detach().numpy()

In [ ]:
torch.save(k, "fpc_cg_tk.pt")

In [ ]:
b.detach().numpy()

In [ ]:
torch.save(b, "fpc_cg_tb.pt")

In [ ]:
train_set = full_set[train_index - 1]
valid_set = full_set[valid_index - 1]
test_set = full_set[test_index - 1]

### The following cell is quite vital, as it indicates the parameters for the **autoencoder**

`input_size` is the number of Node, for example 20550 <br> `dimension` is the dimension of the data in Vtu file, 2 or 3.<br> `components` is the number of components you are compressing, if the tensor is 2 dimension, then it's default of 1. `self_concat` is chennel copying operation, you could keep it default as 1, or 2 when you have multiple SFCs. <br> `structured`: bool, whether you are training SFC-CAE on a structured mesh or not.<br>`nearest_neighboring`: bool, adding sparse layers or not, suggest to turn on.<br>
`dims_letent`: The dimension of the compressed data after applying decoder. <br>
`space_filling_orderings`, `invert_space_filling_orderings` : lists of sfc\inverse_sfc nums, generated by `get_sfc_curves_from_coords` function, or read the orderings from a `csv` file on your own. <br>
`activation`: Can be customly defined, but if you not spcify, the activation would be `ReLU()` for structured mesh, and `Tanh()` for unstructured mesh.

In [ ]:
# input_size = full_tensor.shape[1]
# dimension = 2
# if full_tensor.ndim == 2: components = 1
# else: components = full_tensor.shape[-1]
# self_concat = 1
# structured = False
# nearest_neighbouring = True #True # shut down this may lead to a slightly poorer performance, but a quicker training spped
# dims_latent = 64
# activation = nn.Tanh()
# variational = False
# space_filling_orderings = (mannully spcify it if you didn't generate in the cells before)
# invert_space_filling_orderings = 

### The following are just normal Hyperparameters for training, free to tune it yourself.

In [ ]:
batch_size = 64
seed = 27
lr = 1e-4
n_epochs = 2000
weight_decay = 0
criterion_type = 'MSE'
valid_batch_size = valid_set.shape[0]

In [ ]:
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_set, batch_size=valid_batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=valid_batch_size, shuffle=True)

In [ ]:
coords = torch.Tensor(coords)[:,:2].permute(1,0)

In [ ]:
input_size = full_tensor.shape[1]
dimension = 2
if full_tensor.ndim == 2: components = 1
else: components = full_tensor.shape[-1]
self_concat = 1
structured = False
nearest_neighbouring = True #True # shut down this may lead to a slightly poorer performance, but a quicker training spped
dims_latent = 64
activation = nn.Tanh()
variational = False

In [ ]:
#Create autoencoder 
autoencoder = SFC_CAE(input_size,
                      dimension,
                      components,
                      structured,
                      self_concat,
                      True,
                      dims_latent,
                      space_filling_orderings[:2], 
                      invert_space_filling_orderings[:2],
                      activation,
                      variational = variational,
                      force_initialising_param=None,
                      verbose = False,
                      nfclayers = 0,
                      smoothinglayers = [[(8,33)],[(32,31),(16,27)]])



In [ ]:
# Train
autoencoder = train_model(autoencoder, 
                        train_loader = train_loader,
                        valid_loader = valid_loader,
                        test_loader = test_loader,
                        n_epochs =3000, 
                        lr = 1e-4, 
                        seed = seed,
                        shuffle = True,
                        sfcstoshuffle = space_filling_orderings[:-36],
                        isfcstoshuffle = invert_space_filling_orderings[:-36],
                        changevalid = True,
                        validsfcs = space_filling_orderings[-36:],
                        validisfcs = invert_space_filling_orderings[-36:],
                        visualize = True)

### Other functions need to be add, like convert the result back to `csv`/`vtu` files.....

In [ ]:
# added functions to output vtu files from reconstruct
model_device = torch.device('cuda')
autoencoder = autoencoder.to(model_device)
save_path = 'reconstructed_FPC_CG'
import os
os.system(F'mkdir -p {save_path}')

In [ ]:
data_path = 'FPC_Re3900_CG_new/'
vtu_fields = ['Velocity']

In [ ]:
autoencoder.set_sfcs(space_filling_orderings[930:932],invert_space_filling_orderings[930:932]) #Change the space filling orderings to unseen ones

In [ ]:
# torch.save(autoencoder.state_dict, "FPC_Trained_Model.pth")

In [ ]:
result_vtu_to_vtu(data_path, save_path, vtu_fields, autoencoder, k.detach().numpy(), b.detach().numpy(), model_device = model_device)

Please find the reconstructed files in `reconstructed` folder